In [ ]:
!pip install langchain
!pip install langchain-community # langchain의 일부 document_loaders 기능은 langchain_community 모듈에 있음
!pip install faiss-cpu
!pip install faiss-gpu
!pip install -qU langchain_community faiss-cpu
!pip install pypdf
!pip install sentence_transformers
!pip install rank_bm25
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 427.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5

In [ ]:
from langchain_community.document_loaders import PyPDFLoader # 1.로드 (파일에 따라 TextLoader도 가능)
from langchain.text_splitter import RecursiveCharacterTextSplitter # 2.청크
from langchain.embeddings import HuggingFaceEmbeddings # 3. 임베딩 모델
from langchain.vectorstores import FAISS # 3. 벡터 저장
from langchain.retrievers import BM25Retriever, EnsembleRetriever # 4. 검색 기법
from rank_bm25 import BM25Okapi
import numpy as np

# 1. dense retrieval 사용

In [ ]:
# 1. 문서로딩
loader = PyPDFLoader("한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf")
documents = loader.load()

# 2. 문서를 적절한 크기의 조각으로 청크 (split)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) # # 1000자 크기의 청크로 나누는 예
chunks = text_splitter.split_documents(documents)

# 3. HuggingFaceBgeEmbeddings 사용하여 벡터 임베딩 생성
embedding = HuggingFaceEmbeddings() #model_name="BAAI/bge-small-en-v1.5" 지정 가능

# 3. FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding)

# 4. 질의 실행 및 top-k 문서 검색
query = "위암이면 지급 비용이 어떻게 돼??"
top_k = 3  # 원하는 top-k 문서 개수 설정
results = vectorstore.similarity_search(query, k=top_k)
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
# search_result = retriever.get_relevant_documents(query)


# 결과 출력
print("Top-k Documents:")
for doc in results:
    print(doc.page_content)  # 각 문서의 내용 출력

<ipython-input-6-3ca3f91fef06>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings() #model_name="BAAI/bge-small-en-v1.5" 지정 가능
<ipython-input-6-3ca3f91fef06>:10: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings() #model_name="BAAI/bge-small-en-v1.5" 지정 가능
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in note

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Top-k Documents:
되는 경우 납입최고 (독촉)와 특약의  해지)에서 정한 납입최고 (독촉)기간 내에 갱신 전 계약의  보험료  및 
갱신계약의  제1회 보험료를  납입하지  않으면  이 특약은  갱신되지  않습니다 .
식품의약품안전처  허가범위 
(효능효과 , 용법용량  등) 외 
표적항암제  
식품의약품안전처  허가범 위(효능효과 , 용법용량  등) 
‘암질환심의위원회를  거쳐 
건강보험심사평가원이  승인한 요법’ 인 경우 
‘암질환심의위원회를  거쳐 
건강보험심사평가원이  승인한 요법’ 이 아닌 경우 인정 
인정 
불인정
1. 사망일부 터 사망확인일까지  인출한  소득 
2. 사망확인일  현재 연금계좌에  있는 소득 
 
제107조(장애인의  범위)  
① 법 제51조제1항제2호에 따른 장애인은  다음 각 호의 어느 하나에  해당하는  자로 한다. 
1. 「장애인복지법」에  따른 장애인  및 「장애아동  복지지원법」에  따른 장애아동  중 기획재정부령으


# 2. sparse retrieval 사용

In [ ]:
# 1. PDF 문서 로드
loader = PyPDFLoader("한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf")
documents = loader.load()

# 직접 함수화하여 구현하는 방법(bm25)
# # 각 문서를 토큰화
# def tokenize_documents(documents):
#     return [doc.page_content.split(" ") for doc in documents]

# # BM25 모델을 사용하여 top-k 문서 검색 및 점수 계산
# def search_bm25(query, documents, top_k=5):
#     tokenized_corpus = tokenize_documents(documents)
#     bm25 = BM25Okapi(tokenized_corpus)

#     tokenized_query = query.split(" ")
#     scores = bm25.get_scores(tokenized_query)  # 각 문서에 대한 점수 계산

#     # 점수를 기반으로 top-k 문서 선택
#     top_k_indices = np.argsort(scores)[::-1][:top_k]  # 점수 내림차순으로 top-k 인덱스 선택
#     top_k_docs = [documents[i] for i in top_k_indices]

#     return top_k_docs, scores  # top-k 문서와 점수 반환
# # 질의
# query = "위암이면 지급 비용이 어떻게 돼?"

# # BM25를 사용하여 top-3 문서 검색 및 점수 계산
# top_k_docs, scores = search_bm25(query, documents, top_k=3)

bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 2
results = bm25_retriever.get_relevant_documents(query) # BM25Retriever를 사용하여 query에 맞는 문서 검색

# 검색 결과 출력
for i, doc in enumerate(results):
    print(f"Rank: {i+1}, Content: {doc.page_content}")

Rank: 1, Content: 9 / 531  
 
 보험수익자  또는 
보험가입금액을  
변경하고  싶어요  제26조(계약내용의  변경 등)  
 주소 또는 연락처가  
바뀌었어요  제16조(주소변경통지 )  
 보험나이는  
어떻게  계산하나요  제27조(보험나이 )  
보험계약  
대출 가입한  보험으로  
대출을  받을 수 있나요  제42조(보험계약대출 )  
계약의  
해지 계약을  해지하고  싶어요  제38조(계약자의  임의해지 ) 
제38조의2(위법계약의  해지)  
 해지환급금이  
얼마인지  궁금합니다  제41조(해지환급금 )  
* 주계약  기준 목차로  특약은  특약별  약관 내용을  확인하시기  바랍니다 .
Rank: 2, Content: 19 / 531  
 
  구분 담보명  감액기간  및 비율 
  간편가입  
암수술치료보장  
S특약[특정소액암  
 관혈](갱신형 ) 특정소액암  관혈 수술자금  가입 후 1년간 
보험금  50% 지급 
  간편가입  
암수술치료보장  
S특약[특정소액암   
비관혈 ](갱신형 ) 특정소액암  비관혈  수술자금  가입 후 1년간 
보험금  50% 지급 
  간편가입  
암수술치료보장  
S특약[소액질병  
관혈](갱신형 ) 소액질병  관혈 수술자금  가입 후 1년간 
보험금  50% 지급 
  간편가입  
암수술치료보장  
S특약[소액질병  
비관혈 ](갱신형 ) 소액질병  비관혈  수술자금  가입 후 1년간 
보험금  50% 지급 
  
간편가입  
암다빈치로봇  
수술S특약(갱신형 ) 암(갑상선암 , 전립선암  제외) 수술 
갑상선암 , 전립선암  수술 가입 후 180일간 
보험금  25% 지급 
181일부터  1년간 
보험금  50% 지급 
  간편가입  
유방암수술  
보장S특약 
(갱신형 ) 유방절제수술자금  
유방재건수술자금  가입 후 1년간 
보험금  50% 지급 
  간편가입  
항암 세기조절∙양성자∙
정위적  방사선  
치료S특약(갱신형 ) 항암세기조절방사선치료자금  
항암양성자방사

<ipython-input-7-717a170e0ab8>:31: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = bm25_retriever.get_relevant_documents(query) # BM25Retriever를 사용하여 query에 맞는 문서 검색


# BM25 + LLM 챗봇

In [ ]:
# 1. 문서 로딩 및 청크 분할
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf")  # PDF 문서 경로로 수정
documents = loader.load()

# 문서를 적절한 크기로 청크 나누기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

# 2. BM25 Retriever 설정
from langchain.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(chunks)

# 3. LLM과 결합하여 답변 생성
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 모델과 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained("beomi/Llama-3-Open-Ko-8B")
model = AutoModelForCausalLM.from_pretrained("beomi/Llama-3-Open-Ko-8B")

# 파이프라인 생성
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# 4. BM25 기반 검색과 LLM 사용하여 RAG 동작
def query_bm25(query):
    # BM25로 관련 문서 검색
    results = bm25_retriever.get_relevant_documents(query)

    # 검색된 문서 출력 (디버깅용)
    for i, doc in enumerate(results):
        print(f"Rank: {i+1}, Content: {doc.page_content}")

    # 검색된 문서들로부터 하나의 문맥 구성
    context = " ".join([doc.page_content for doc in results])

    # LLM을 사용하여 답변 생성
    llm_response = qa_pipeline(question=query, context=context)

    return llm_response['answer']

# 5. Gradio 인터페이스 생성
import gradio as gr

# Gradio에서 query를 입력받아 chat 함수에 전달
def chat(query):
    response = query_bm25(query)  # Gradio 입력을 query로 처리
    return response

# Gradio 인터페이스 생성
iface = gr.Interface(fn=chat, inputs="text", outputs="text", title="BM25 + LLM 기반 RAG 챗봇")
iface.launch()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

...

model-00003-of-00006.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]